In [6]:
import torch
print(torch.__version__)

2.7.0+cu118


In [7]:
!nvidia-smi

Fri May 23 21:55:52 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 572.83                 Driver Version: 572.83         CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3060      WDDM  |   00000000:01:00.0  On |                  N/A |
|  0%   35C    P5             19W /  170W |     944MiB /  12288MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [16]:
from groundingdino.util.inference import load_model, load_image, predict, annotate
import cv2
import os
import torch

# Función para unificar etiquetas similares
def unify_label(label):
    if label in ["onion", "frozen onion bag"]:
        return "cebolla"
    return label

# Configuraciones
MODEL_CONFIG = "GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py"
MODEL_WEIGHT = "GroundingDINO/weights/groundingdino_swint_ogc.pth"
IMAGE_FOLDER = "C:/Users/Lucas/Downloads/datatrain/cebolla"
OUTPUT_FOLDER = "C:/Users/Lucas/Downloads/datatrain/cebolla/output"

TEXT_PROMPT = "onion . frozen onion bag"  # pone aquí tus etiquetas
BOX_THRESHOLD = 0.35
TEXT_THRESHOLD = 0.25

# Asegurarse que la carpeta de salida existe
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# Cargar modelo (automáticamente usa GPU si está disponible)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Usando dispositivo: {device}")

model = load_model(MODEL_CONFIG, MODEL_WEIGHT, device=device)

#ejecutar con cpu
model = load_model(MODEL_CONFIG, MODEL_WEIGHT, device="cpu")

# Procesar todas las imágenes en la carpeta
for filename in os.listdir(IMAGE_FOLDER):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
        image_path = os.path.join(IMAGE_FOLDER, filename)
        print(f"Procesando {image_path}")

        image_source, image = load_image(image_path)

        boxes, logits, phrases = predict(
            model=model,
            image=image,
            caption=TEXT_PROMPT,
            box_threshold=BOX_THRESHOLD,
            text_threshold=TEXT_THRESHOLD
        )

        # Unificar etiquetas detectadas
        phrases = [unify_label(p) for p in phrases]

        annotated_frame = annotate(image_source=image_source, boxes=boxes, logits=logits, phrases=phrases)

        output_path = os.path.join(OUTPUT_FOLDER, f"annotated_{filename}")
        cv2.imwrite(output_path, annotated_frame)
        print(f"Imagen guardada en {output_path}")

print("Procesamiento completado.")

Usando dispositivo: cuda
final text_encoder_type: bert-base-uncased
final text_encoder_type: bert-base-uncased
Procesando C:/Users/Lucas/Downloads/datatrain/cebolla\000000000000662577-UN-01.jpg


NameError: name '_C' is not defined